# **Setup and Library Imports**

### **Installing Required Packages**

In [1]:
! pip install --quiet "transformers[torch]"
! pip install --quiet evaluate
! pip install --quiet tabulate
! pip install --quiet ipywidgets
! pip install --quiet datasets
! pip install --quiet pillow
! pip install --quiet scikit-learn
! pip install --quiet tensorboard

   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 193.5/363.4 MB 212.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.2 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone 

### **Connect to google drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Logging into Hugging Face Hub**

In [ ]:
# from huggingface_hub import notebook_login
## Execute the login function to access the Hugging Face account
# notebook_login()

### **Importing Libraries**

In [3]:
# PyTorch for tensor operations
import torch

# Hugging Face libraries for training and transformer models
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import AdamW

# Evaluation metrics and utilities
import evaluate
import numpy as np
from datetime import datetime

# Loading datasets for training and evaluation
from datasets import load_dataset

# Data manipulation and display utilities
import pandas as pd
from tabulate import tabulate
from collections import Counter

### **Defining Model, Dataset Paths, and Output Directories**

List of Models


```
microsoft/resnet-50
google/vit-base-patch16-224
microsoft/swin-base-patch4-window7-224
facebook/deit-base-patch16-224
microsoft/beit-base-patch16-224
facebook/dinov2-base
```



List of Datasets


```
cvmil/rice-leaf-disease-augmented-v3
cvmil/rice-leaf-disease-augmented-v2
cvmil/rice-leaf-disease-augmented
cvmil/rice-leaf-disease-augmented-test
cvmil/rice-disease-02
```

Define paths for saving model training outputs and logs, incorporating model and dataset names along with the current date.

In [4]:
# Define model and dataset paths
model_path = "microsoft/swin-base-patch4-window7-224"
dataset_path = "cvmil/rice-leaf-disease-augmented-v3"
train_epochs = 20
resume_from_checkpoint = False

base_model_name = model_path.split("/")[-1]
dataset_name = dataset_path.split("/")[-1]

model_name = f"test_{base_model_name}_{dataset_name}_tl"
output_dir = f"./drive/Shareddrives/CS198-Drones/final_training_output/{model_name}"

# Define directory for storing training logs
logging_dir = f"{output_dir}/logs"

# **Data Preparation and Processing Pipeline**

This section handles the dataset loading, label extraction, image processing setup, and defines necessary functions for data transformation, batching, and metric computation to prepare the data for model training and evaluation.

### **Load Dataset and Extract Labels**

Load the dataset from huggingface and extract the class labels from the training data.

In [5]:
# Load the dataset
dataset = load_dataset(dataset_path)

# Extract class labels from the training set
labels = dataset['train'].features['label'].names

README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/354M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/558M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/363M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/328M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/55.4M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8192 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/307 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/303 [00:00<?, ? examples/s]

Generate and display a table showing class distribution across training and validation splits.

In [6]:
label_mapping = dataset['train'].features['label'].int2str

# Count the number of samples per class in each split
train_counts = Counter(dataset['train']['label'])
validation_counts = Counter(dataset['validation']['label'])

# Create a DataFrame for the class distribution
data = {
    'ID': list(range(len(labels))),
    'Label': labels,
    'Training': [train_counts[i] if i in train_counts else 0 for i in range(len(labels))],
    'Validation': [validation_counts[i] if i in validation_counts else 0 for i in range(len(labels))],
}

# Display the class distribution in a table format
df = pd.DataFrame(data)
print(tabulate(df, headers='keys', tablefmt='grid', showindex=False))

+------+------------------------+------------+--------------+
|   ID | Label                  |   Training |   Validation |
+======+========================+============+==============+
|    0 | Bacterial Leaf Blight  |       1024 |           34 |
+------+------------------------+------------+--------------+
|    1 | Brown Spot             |       1024 |           50 |
+------+------------------------+------------+--------------+
|    2 | Healthy Rice Leaf      |       1024 |           30 |
+------+------------------------+------------+--------------+
|    3 | Leaf Blast             |       1024 |           42 |
+------+------------------------+------------+--------------+
|    4 | Leaf scald             |       1024 |           35 |
+------+------------------------+------------+--------------+
|    5 | Narrow Brown Leaf Spot |       1024 |           22 |
+------+------------------------+------------+--------------+
|    6 | Rice Hispa             |       1024 |           41 |
+------+

### **Initialize Image Processor**

Load and initialize the image processor from the pre-trained model.

In [7]:
# Load the image processor from the pre-trained model
processor = AutoImageProcessor.from_pretrained(model_path)
print(processor)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


ViTImageProcessor {
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



### **Data Preparation and Processing Pipeline**

Create mappings for label-to-ID and ID-to-label.

In [8]:
label2id = {c: idx for idx, c in enumerate(labels)}
id2label = {idx: c for idx, c in enumerate(labels)}

Define the transformation function to process the image batch.

In [9]:
def transforms(batch):
    batch['image'] = [x.convert('RGB') for x in batch['image']]
    inputs = processor(batch['image'], return_tensors='pt')
    inputs['labels'] = batch['label']
    return inputs

Define the custom collation function for batching pixel values and labels.

In [10]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }


Define the function to compute accuracy during evaluation.

In [11]:
accuracy = evaluate.load('accuracy')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)


### **Apply Data Transformations to Dataset**

Apply the defined transformation function to the dataset for preprocessing. </br>
Note: This assumes that data augmentation and normalization have already been handled in the previous pipeline and is ready for fine-tuning.

In [12]:
processed_dataset = dataset.with_transform(transforms)

# **Model Initialization and Trainer Setup**

This section handles the initialization of the model, configuration of training parameters, and setting up the Trainer for fine-tuning, including the datasets, data processing, and evaluation metrics.

### **Initialize Pre-trained Model for Fine-tuning**

Load a pre-trained image classification model, configuring it with the correct label mappings and number of labels for the fine-tuning task.

In [13]:
# Load pre-trained model and configure it for fine-tuning
model = AutoModelForImageClassification.from_pretrained(
    model_path,                  # Path to the pre-trained model
    num_labels=len(labels),      # Set the number of labels for classification
    id2label=id2label,           # Map from ID to label
    label2id=label2id,           # Map from label to ID
    ignore_mismatched_sizes=True # Ignore size mismatches in weights
)

config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/352M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-base-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([8]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 1024]) in the checkpoint and torch.Size([8, 1024]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### **Check Model Parameters for Fine-tuning**

Freeze the model parameters except for those related to the classifier layers, allowing fine-tuning only on the specified layers.

In [14]:
for name, param in model.named_parameters():
    param.requires_grad = name.startswith(('classifier', 'distillation_classifier'))

We can check how many parameters are there in the model along with how many are actually going to be trained now.

In [15]:
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {num_params:,} | Trainable parameters: {trainable_params:,}")

Total parameters: 86,751,424 | Trainable parameters: 8,200


### **Define Training Arguments**

Set up the training configuration with parameters such as batch size, number of epochs, learning rate, and logging strategies for the fine-tuning process.

In [19]:
training_args = TrainingArguments(
    num_train_epochs=train_epochs,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,

    fp16 = True,
    warmup_ratio=0.1,
    weight_decay=0.05,

    learning_rate=3e-4,
    lr_scheduler_type="cosine_with_restarts",
    lr_scheduler_kwargs={"num_cycles": 4},

    save_total_limit=3,
    report_to=['tensorboard'],
    save_strategy="epoch",
    eval_strategy="epoch",
    logging_strategy="epoch",
    logging_dir=logging_dir,
    output_dir=output_dir,

    remove_unused_columns=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    push_to_hub=True,
)

### **Initialize Trainer**

Initialize the Trainer object with the model, training arguments, data collator, metrics computation, and datasets for training and evaluation.

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor,
)

<ipython-input-20-bc637b425f01>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### **Trainer Callback**

In [21]:
class CustomSaveCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        trainer.save_model()
        trainer.save_state()

In [22]:
trainer.add_callback(CustomSaveCallback)

### **Create Model Card**

In [23]:
trainer.create_model_card(
    language="en",
    license="MIT",
    tags=["image-classification", "fine-tuning"],
    model_name=model_name,
    finetuned_from=base_model_name,
    tasks=["image-classification"],
    dataset_tags=["image", "rice-leaf-disease"],
    dataset=dataset_name,
    dataset_args=["size: 224x224", "augmentation: true"],
)

# **Model Training and Evaluation**

### **Start Fine-tuning Process**

Initiates the fine-tuning of the model using the Trainer, applying the specified training configurations, such as the batch size, learning rate, and number of epochs. During training, the model will be evaluated at the end of each epoch on the validation dataset using the compute_metrics function, which calculates accuracy.

The model will undergo the following process during fine-tuning:
- **Training**: The model will be trained on the training dataset for the specified number of epochs.
- **Evaluation**: After each epoch, the model will be evaluated on the validation dataset, and accuracy will be computed using the compute_metrics function.
- **Metrics Logging**: The training progress and evaluation results will be logged to TensorBoard and can be monitored during training.

In [ ]:
print(f"Training {model_name} on {dataset_name} dataset...")
train_results = trainer.train(resume_from_checkpoint=resume_from_checkpoint)

Training test_swin-base-patch4-window7-224_rice-leaf-disease-augmented-v3_tl on rice-leaf-disease-augmented-v3 dataset...


Epoch,Training Loss,Validation Loss,Accuracy
1,1.882900,1.483645,0.576547


### **Save Model and Training State**

After the training process, the model and relevant training state are saved. This includes saving the model weights, training metrics, and the state of the trainer, ensuring that training progress can be restored if needed.

In [ ]:
# Save the trained model
trainer.save_model()

# Log and save training metrics for later reference
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)

# Save the state of the trainer, including configuration and optimizer state
trainer.save_state()